This module is for __HOURLY OHLCV__ data, set at __48__ hours:

    ** 1. Merge the pumped signals with time stamps to get vertical format for voting
    ** 2. Append the output whethere the coin was pumped or not (binary 0)

In [30]:
import os, sys
import pandas as pd 
import numpy as np
import requests 
from datetime import datetime, date, time, timedelta
import pytz
import matplotlib.pyplot as plt
import matplotlib.dates as mdate
from mpl_finance import candlestick_ohlc
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.ticker as mticker
import pickle
pd.set_option('display.max_columns', 999)
%load_ext autoreload
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Import the pumped data

### Step 1. Create the timestamp for the first hours 

In [31]:
p = pd.read_pickle("../all_pumps.pkl")
# p = p[p.currency == '***'] #Experiment for borderline check
# p = p[['channelLink','channelTitle','currency','exchange','signalTime','pumptime_edited','timestamp']]
print(p.shape)
p.head(2) 

(59286, 7)


,currency,exchange,pumptime_edited,coin_name,timestamp,event_date,event_time
0,NAV,Binance,2019-12-19 17:00:15+00:00,VIB,1576774815,2019-12-19,17:00:15
1,NAV,Binance,2019-12-19 17:00:15+00:00,STEEM,1576774815,2019-12-19,17:00:15


Ther are .... unique points in time...

In [32]:
#Number of unique time points
len(pd.unique(p['timestamp']))

481

In [33]:
#Number of unique coins points
len(pd.unique(p['currency']))

109

In [34]:
#Number of unique currency allowed
len(pd.unique(p['coin_name']))

123

In [35]:
3600*24

86400

In [36]:
1562346032 - (1562346032 % (86400))

1562284800

..Strange beheavior, must write each function one by one...

In [37]:
def getHourstamp(dfo: pd.DataFrame, num_hours: int=1, var:str='timestamp'):
    '''
    Append a column that contains the timestamp at 12:00 AM (0:00) of the
    original var variable.
    :param df: dataframe in question 
    :param num_date: number of dates to go back to
    :param var str: name of the variable that has original timestamp
    '''
    df = dfo.copy()
    df[var+'_int'] = df[var].apply(int)
    for d in range(num_hours+1):
        df[var+'_h'+str(d)] = df[var+'_int'].apply(lambda t : t - (t % (3600)) - d*(3600))
    df = df.drop(columns=var+'_int')
    return df

In [38]:
num_hours = 48
p = getHourstamp(p,num_hours)
p.sample(5)

,currency,exchange,pumptime_edited,coin_name,timestamp,event_date,event_time,timestamp_h0,timestamp_h1,timestamp_h2,timestamp_h3,timestamp_h4,timestamp_h5,timestamp_h6,timestamp_h7,timestamp_h8,timestamp_h9,timestamp_h10,timestamp_h11,timestamp_h12,timestamp_h13,timestamp_h14,timestamp_h15,timestamp_h16,timestamp_h17,timestamp_h18,timestamp_h19,timestamp_h20,timestamp_h21,timestamp_h22,timestamp_h23,timestamp_h24,timestamp_h25,timestamp_h26,timestamp_h27,timestamp_h28,timestamp_h29,timestamp_h30,timestamp_h31,timestamp_h32,timestamp_h33,timestamp_h34,timestamp_h35,timestamp_h36,timestamp_h37,timestamp_h38,timestamp_h39,timestamp_h40,timestamp_h41,timestamp_h42,timestamp_h43,timestamp_h44,timestamp_h45,timestamp_h46,timestamp_h47,timestamp_h48
16578,***,Binance,2019-11-07 01:00:00+00:00,ENG,1573088400,2019-11-07,01:00:00,1573088400,1573084800,1573081200,1573077600,1573074000,1573070400,1573066800,1573063200,1573059600,1573056000,1573052400,1573048800,1573045200,1573041600,1573038000,1573034400,1573030800,1573027200,1573023600,1573020000,1573016400,1573012800,1573009200,1573005600,1573002000,1572998400,1572994800,1572991200,1572987600,1572984000,1572980400,1572976800,1572973200,1572969600,1572966000,1572962400,1572958800,1572955200,1572951600,1572948000,1572944400,1572940800,1572937200,1572933600,1572930000,1572926400,1572922800,1572919200,1572915600
38571,STORM,Binance,2018-09-23 11:01:21+00:00,XZC,1537700481,2018-09-23,11:01:21,1537700400,1537696800,1537693200,1537689600,1537686000,1537682400,1537678800,1537675200,1537671600,1537668000,1537664400,1537660800,1537657200,1537653600,1537650000,1537646400,1537642800,1537639200,1537635600,1537632000,1537628400,1537624800,1537621200,1537617600,1537614000,1537610400,1537606800,1537603200,1537599600,1537596000,1537592400,1537588800,1537585200,1537581600,1537578000,1537574400,1537570800,1537567200,1537563600,1537560000,1537556400,1537552800,1537549200,1537545600,1537542000,1537538400,1537534800,1537531200,1537527600
37810,OAX,Binance,2018-10-09 19:00:48+00:00,NXS,1539111648,2018-10-09,19:00:48,1539111600,1539108000,1539104400,1539100800,1539097200,1539093600,1539090000,1539086400,1539082800,1539079200,1539075600,1539072000,1539068400,1539064800,1539061200,1539057600,1539054000,1539050400,1539046800,1539043200,1539039600,1539036000,1539032400,1539028800,1539025200,1539021600,1539018000,1539014400,1539010800,1539007200,1539003600,1539000000,1538996400,1538992800,1538989200,1538985600,1538982000,1538978400,1538974800,1538971200,1538967600,1538964000,1538960400,1538956800,1538953200,1538949600,1538946000,1538942400,1538938800
28191,EDO,Binance,2019-03-12 17:39:12+00:00,WPR,1552412352,2019-03-12,17:39:12,1552410000,1552406400,1552402800,1552399200,1552395600,1552392000,1552388400,1552384800,1552381200,1552377600,1552374000,1552370400,1552366800,1552363200,1552359600,1552356000,1552352400,1552348800,1552345200,1552341600,1552338000,1552334400,1552330800,1552327200,1552323600,1552320000,1552316400,1552312800,1552309200,1552305600,1552302000,1552298400,1552294800,1552291200,1552287600,1552284000,1552280400,1552276800,1552273200,1552269600,1552266000,1552262400,1552258800,1552255200,1552251600,1552248000,1552244400,1552240800,1552237200
15196,INS,Binance,2019-06-25 16:00:08+00:00,BCH,1561478408,2019-06-25,16:00:08,1561478400,1561474800,1561471200,1561467600,1561464000,1561460400,1561456800,1561453200,1561449600,1561446000,1561442400,1561438800,1561435200,1561431600,1561428000,1561424400,1561420800,1561417200,1561413600,1561410000,1561406400,1561402800,1561399200,1561395600,1561392000,1561388400,1561384800,1561381200,1561377600,1561374000,1561370400,1561366800,1561363200,1561359600,1561356000,1561352400,1561348800,1561345200,1561341600,1561338000,1561334400,1561330800,1561327200,1561323600,1561320000,1561316400,1561312800,1561309200,1561305600


### Test an Processed Satoshi dataset before Merging

In [39]:
# s = pd.read_pickle("../../Coin_Data/Hourly/Processed_Data/Sat_Hourly_ADX_at_Bittrex.pkl")

In [40]:
# s.head()

In [41]:
fin_vars = ['open','high','low','close','volumefrom','volumeto','date','date_utc']

### Step 2: Loop throug the processed social data file and merge based on timestemp
... We do not need to join by the coinID nor coin name. 

Just the __timestamps__ and __EXCHANGE__ are good enough. 
Since we want to retrieve the values for all available timestamps.

## HOURLY

In [42]:
 ## Check what's inside the downoaded file
dl = pd.read_pickle("../../Coin_Data/Hourly/Hourly_VIB_at_Binance.pkl")
dl['dt']= dl.date_utc.apply(lambda d:d.date)
dl.head(2)

,time,close,high,low,open,volumefrom,volumeto,exchange,date,date_utc,dt
0,1520154000,0.000023,0.000023,0.000023,0.000023,85742,1.972,Binance,2018-03-04 01:00:00,2018-03-04 09:00:00+00:00,2018-03-04
1,1520157600,0.000023,0.000023,0.000023,0.000023,43869,1.003,Binance,2018-03-04 02:00:00,2018-03-04 10:00:00+00:00,2018-03-04


In [43]:
dl[dl.time == 1577757600]

,time,close,high,low,open,volumefrom,volumeto,exchange,date,date_utc,dt


... Run the merge for all available daily data...

In [44]:
dir_prefix = "../../Coin_Data/Hourly/"
ohlc_error = []
x_final = pd.DataFrame()
t_temp = pd.DataFrame()
cnt = 0

for filename in os.listdir(dir_prefix):
    if 'pkl' in filename and 'error' not in filename:
#         print(filename)
        try: 
            _,coin_name,_,exchange = filename.split('_')
            exchange = exchange[:-4]
#             print(exchange)
            print("Processing file...",filename)
            #Make sure 
            l = p[(p.coin_name == coin_name) & (p.exchange == exchange)].copy()
#             print(l.shape)
            for d in range(num_hours+1):
                if 'timestamp_h'+str(d) not in l.columns: 
                    print('Info. on hour_{} not in the left table!'.format(d))
                    continue
                x = pd.read_pickle(dir_prefix+filename)
                x.columns = [c+'_h'+str(d) if c != 'time' else 'time' for c in x.columns]
                #Merge the data based on time and exchange
                l = pd.merge(left=l
                             ,left_on=['timestamp_h'+str(d),'exchange']
                             ,right=x, right_on=["time","exchange_h"+str(d)]
                             ,how='inner')
#                 print(l.shape)
                l = l.drop(columns=['time',"exchange_h"+str(d)])
            #Save the file in the processed_directory
            l['coin_name'] = coin_name
            #Append all the data on top of each other for processing; 
            x_final = x_final.append(l)
#             del l
        except: 
            e = sys.exc_info()[0]
            print( "<p>Error: %s</p>" % e )
            ohlc_error.append(filename)
            continue
#     cnt += 1 
#     if cnt >= 1:
#         break

Processing file... Hourly_VIB_at_Binance.pkl
Processing file... Hourly_STEEM_at_Binance.pkl
Processing file... Hourly_NEBL_at_Binance.pkl
Processing file... Hourly_WABI_at_Binance.pkl
Processing file... Hourly_FTM_at_Binance.pkl
Processing file... Hourly_ONT_at_Binance.pkl
Processing file... Hourly_FET_at_Binance.pkl
Processing file... Hourly_XZC_at_Binance.pkl
Processing file... Hourly_GVT_at_Binance.pkl
Processing file... Hourly_SYS_at_Binance.pkl
Processing file... Hourly_LINK_at_Binance.pkl
Processing file... Hourly_DNT_at_Binance.pkl
Processing file... Hourly_QKC_at_Binance.pkl
Processing file... Hourly_THETA_at_Binance.pkl
Processing file... Hourly_CDT_at_Binance.pkl
Processing file... Hourly_FUN_at_Binance.pkl
Processing file... Hourly_ELC_at_Binance.pkl
Processing file... Hourly_EOS_at_Binance.pkl
Processing file... Hourly_OST_at_Binance.pkl
Processing file... Hourly_TNT_at_Binance.pkl
Processing file... Hourly_NEO_at_Binance.pkl
Processing file... Hourly_QSP_at_Binance.pkl
Pro

In [45]:
print(ohlc_error)
# print(l.shape)

[]


#We might have to check whether there is a non-zero data somewhere....

In [46]:
print(x_final.shape)
print(x_final.drop_duplicates().shape)
x_final.head(3)

(60393, 448)
(59286, 448)


,currency,exchange,pumptime_edited,coin_name,timestamp,event_date,event_time,timestamp_h0,timestamp_h1,timestamp_h2,timestamp_h3,timestamp_h4,timestamp_h5,timestamp_h6,timestamp_h7,timestamp_h8,timestamp_h9,timestamp_h10,timestamp_h11,timestamp_h12,timestamp_h13,timestamp_h14,timestamp_h15,timestamp_h16,timestamp_h17,timestamp_h18,timestamp_h19,timestamp_h20,timestamp_h21,timestamp_h22,timestamp_h23,timestamp_h24,timestamp_h25,timestamp_h26,timestamp_h27,timestamp_h28,timestamp_h29,timestamp_h30,timestamp_h31,timestamp_h32,timestamp_h33,timestamp_h34,timestamp_h35,timestamp_h36,timestamp_h37,timestamp_h38,timestamp_h39,timestamp_h40,timestamp_h41,timestamp_h42,timestamp_h43,timestamp_h44,timestamp_h45,timestamp_h46,timestamp_h47,timestamp_h48,close_h0,high_h0,low_h0,open_h0,volumefrom_h0,volumeto_h0,date_h0,date_utc_h0,close_h1,high_h1,low_h1,open_h1,volumefrom_h1,volumeto_h1,date_h1,date_utc_h1,close_h2,high_h2,low_h2,open_h2,volumefrom_h2,volumeto_h2,date_h2,date_utc_h2,close_h3,high_h3,low_h3,open_h3,volumefrom_h3,volumeto_h3,date_h3,date_utc_h3,close_h4,high_h4,low_h4,open_h4,volumefrom_h4,volumeto_h4,date_h4,date_utc_h4,close_h5,high_h5,low_h5,open_h5,volumefrom_h5,volumeto_h5,date_h5,date_utc_h5,close_h6,high_h6,low_h6,open_h6,volumefrom_h6,volumeto_h6,date_h6,date_utc_h6,close_h7,high_h7,low_h7,open_h7,volumefrom_h7,volumeto_h7,date_h7,date_utc_h7,close_h8,high_h8,low_h8,open_h8,volumefrom_h8,volumeto_h8,date_h8,date_utc_h8,close_h9,high_h9,low_h9,open_h9,volumefrom_h9,volumeto_h9,date_h9,date_utc_h9,close_h10,high_h10,low_h10,open_h10,volumefrom_h10,volumeto_h10,date_h10,date_utc_h10,close_h11,high_h11,low_h11,open_h11,volumefrom_h11,volumeto_h11,date_h11,date_utc_h11,close_h12,high_h12,low_h12,open_h12,volumefrom_h12,volumeto_h12,date_h12,date_utc_h12,close_h13,high_h13,low_h13,open_h13,volumefrom_h13,volumeto_h13,date_h13,date_utc_h13,close_h14,high_h14,low_h14,open_h14,volumefrom_h14,volumeto_h14,date_h14,date_utc_h14,close_h15,high_h15,low_h15,open_h15,volumefrom_h15,volumeto_h15,date_h15,date_utc_h15,close_h16,high_h16,low_h16,open_h16,volumefrom_h16,volumeto_h16,date_h16,date_utc_h16,close_h17,high_h17,low_h17,open_h17,volumefrom_h17,volumeto_h17,date_h17,date_utc_h17,close_h18,high_h18,low_h18,open_h18,volumefrom_h18,volumeto_h18,date_h18,date_utc_h18,close_h19,high_h19,low_h19,open_h19,volumefrom_h19,volumeto_h19,date_h19,date_utc_h19,close_h20,high_h20,low_h20,open_h20,volumefrom_h20,volumeto_h20,date_h20,date_utc_h20,close_h21,high_h21,low_h21,open_h21,volumefrom_h21,volumeto_h21,date_h21,date_utc_h21,close_h22,high_h22,low_h22,open_h22,volumefrom_h22,volumeto_h22,date_h22,date_utc_h22,close_h23,high_h23,low_h23,open_h23,volumefrom_h23,volumeto_h23,date_h23,date_utc_h23,close_h24,high_h24,low_h24,open_h24,volumefrom_h24,volumeto_h24,date_h24,date_utc_h24,close_h25,high_h25,low_h25,open_h25,volumefrom_h25,volumeto_h25,date_h25,date_utc_h25,close_h26,high_h26,low_h26,open_h26,volumefrom_h26,volumeto_h26,date_h26,date_utc_h26,close_h27,high_h27,low_h27,open_h27,volumefrom_h27,volumeto_h27,date_h27,date_utc_h27,close_h28,high_h28,low_h28,open_h28,volumefrom_h28,volumeto_h28,date_h28,date_utc_h28,close_h29,high_h29,low_h29,open_h29,volumefrom_h29,volumeto_h29,date_h29,date_utc_h29,close_h30,high_h30,low_h30,open_h30,volumefrom_h30,volumeto_h30,date_h30,date_utc_h30,close_h31,high_h31,low_h31,open_h31,volumefrom_h31,volumeto_h31,date_h31,date_utc_h31,close_h32,high_h32,low_h32,open_h32,volumefrom_h32,volumeto_h32,date_h32,date_utc_h32,close_h33,high_h33,low_h33,open_h33,volumefrom_h33,volumeto_h33,date_h33,date_utc_h33,close_h34,high_h34,low_h34,open_h34,volumefrom_h34,volumeto_h34,date_h34,date_utc_h34,close_h35,high_h35,low_h35,open_h35,volumefrom_h35,volumeto_h35,date_h35,date_utc_h35,close_h36,high_h36,low_h36,open_h36,volumefrom_h36,volumeto_h36,date_h36,date_utc_h36,close_h37,high_h37,low_h37,open_h37,volumefrom_h37,volumeto_h37,date_h37,date_utc_h37,close_h38,high_h38,low_h38,open_h38,volumefrom_h38,volumeto

## CHECK MERGE FOR CORRECT INFORMATION

In [47]:
a = p[["currency","pumptime_edited","coin_name","timestamp"]]

In [48]:
b = x_final[["currency","pumptime_edited","coin_name","timestamp"]]

In [49]:
c = pd.merge(left=a,left_on=['coin_name','pumptime_edited']
             ,right=b,right_on=['coin_name','pumptime_edited'],how="left",suffixes=('_a','_b'))
c.head(2)

,currency_a,pumptime_edited,coin_name,timestamp_a,currency_b,timestamp_b
0,NAV,2019-12-19 17:00:15+00:00,VIB,1576774815,NAV,1576774815
1,NAV,2019-12-19 17:00:15+00:00,STEEM,1576774815,NAV,1576774815


In [50]:
na = c[pd.isna(c.currency_b) & (c.coin_name != 'error') & (c.coin_name != 'OHLCV')]
print(na.shape)

(0, 6)


In [51]:
na.drop_duplicates()

,currency_a,pumptime_edited,coin_name,timestamp_a,currency_b,timestamp_b


## CLEAN UP AND SAVE 

In [52]:
x_final = x_final.reset_index(drop=True)
x_final = x_final.sort_values(['timestamp','exchange'])
x_final = x_final.drop_duplicates()

In [53]:
x_final.filter(regex="open").head()

,open_h0,open_h1,open_h2,open_h3,open_h4,open_h5,open_h6,open_h7,open_h8,open_h9,open_h10,open_h11,open_h12,open_h13,open_h14,open_h15,open_h16,open_h17,open_h18,open_h19,open_h20,open_h21,open_h22,open_h23,open_h24,open_h25,open_h26,open_h27,open_h28,open_h29,open_h30,open_h31,open_h32,open_h33,open_h34,open_h35,open_h36,open_h37,open_h38,open_h39,open_h40,open_h41,open_h42,open_h43,open_h44,open_h45,open_h46,open_h47,open_h48
330,0.000017,0.000017,0.000017,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000017,0.000017,0.000018,0.000017,0.000018,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000018,0.000018,0.000018
821,0.000320,0.000319,0.000321,0.000321,0.000319,0.000317,0.000316,0.000316,0.000316,0.000318,0.000318,0.000322,0.000320,0.000318,0.000324,0.000323,0.000322,0.000318,0.000321,0.000319,0.000320,0.000318,0.000312,0.000311,0.000312,0.000312,0.000309,0.000313,0.000314,0.000313,0.000317,0.000312,0.000315,0.000314,0.000314,0.000318,0.000318,0.000317,0.000316,0.000311,0.000311,0.000310,0.000309,0.000311,0.000313,0.000313,0.000320,0.000321,0.000322
1312,0.001030,0.001037,0.001026,0.001038,0.001038,0.001041,0.001027,0.001039,0.001045,0.001035,0.001039,0.001040,0.001045,0.001037,0.001056,0.001042,0.001039,0.001033,0.001046,0.001046,0.001050,0.001042,0.001041,0.001054,0.001045,0.001050,0.001047,0.001037,0.001043,0.001042,0.001050,0.001017,0.001010,0.001017,0.001022,0.001025,0.001020,0.001008,0.001004,0.001011,0.000996,0.000999,0.001005,0.000994,0.001000,0.001000,0.001028,0.001046,0.001052
1803,0.000100,0.000101,0.000101,0.000102,0.000102,0.000102,0.000102,0.000103,0.000104,0.000103,0.000104,0.000104,0.000103,0.000102,0.000104,0.000104,0.000103,0.000103,0.000103,0.000103,0.000102,0.000102,0.000104,0.000105,0.000105,0.000103,0.000102,0.000101,0.000103,0.000098,0.000097,0.000096,0.000095,0.000095,0.000095,0.000095,0.000094,0.000094,0.000094,0.000093,0.000092,0.000092,0.000092,0.000093,0.000093,0.000093,0.000094,0.000095,0.000096
2294,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


__!!__ Notice that we have only __rougly half__ the number of obs. as the social media only case, 
but the additional constraints of exhange explains that (Few coins listed at each exchange).

*** Now we have the time series data up to 48 horus in the past!
*** So now we can start preparing the data and sort the categories together?

In [54]:
print(x_final.shape)
x_final.head()

(59286, 448)


,currency,exchange,pumptime_edited,coin_name,timestamp,event_date,event_time,timestamp_h0,timestamp_h1,timestamp_h2,timestamp_h3,timestamp_h4,timestamp_h5,timestamp_h6,timestamp_h7,timestamp_h8,timestamp_h9,timestamp_h10,timestamp_h11,timestamp_h12,timestamp_h13,timestamp_h14,timestamp_h15,timestamp_h16,timestamp_h17,timestamp_h18,timestamp_h19,timestamp_h20,timestamp_h21,timestamp_h22,timestamp_h23,timestamp_h24,timestamp_h25,timestamp_h26,timestamp_h27,timestamp_h28,timestamp_h29,timestamp_h30,timestamp_h31,timestamp_h32,timestamp_h33,timestamp_h34,timestamp_h35,timestamp_h36,timestamp_h37,timestamp_h38,timestamp_h39,timestamp_h40,timestamp_h41,timestamp_h42,timestamp_h43,timestamp_h44,timestamp_h45,timestamp_h46,timestamp_h47,timestamp_h48,close_h0,high_h0,low_h0,open_h0,volumefrom_h0,volumeto_h0,date_h0,date_utc_h0,close_h1,high_h1,low_h1,open_h1,volumefrom_h1,volumeto_h1,date_h1,date_utc_h1,close_h2,high_h2,low_h2,open_h2,volumefrom_h2,volumeto_h2,date_h2,date_utc_h2,close_h3,high_h3,low_h3,open_h3,volumefrom_h3,volumeto_h3,date_h3,date_utc_h3,close_h4,high_h4,low_h4,open_h4,volumefrom_h4,volumeto_h4,date_h4,date_utc_h4,close_h5,high_h5,low_h5,open_h5,volumefrom_h5,volumeto_h5,date_h5,date_utc_h5,close_h6,high_h6,low_h6,open_h6,volumefrom_h6,volumeto_h6,date_h6,date_utc_h6,close_h7,high_h7,low_h7,open_h7,volumefrom_h7,volumeto_h7,date_h7,date_utc_h7,close_h8,high_h8,low_h8,open_h8,volumefrom_h8,volumeto_h8,date_h8,date_utc_h8,close_h9,high_h9,low_h9,open_h9,volumefrom_h9,volumeto_h9,date_h9,date_utc_h9,close_h10,high_h10,low_h10,open_h10,volumefrom_h10,volumeto_h10,date_h10,date_utc_h10,close_h11,high_h11,low_h11,open_h11,volumefrom_h11,volumeto_h11,date_h11,date_utc_h11,close_h12,high_h12,low_h12,open_h12,volumefrom_h12,volumeto_h12,date_h12,date_utc_h12,close_h13,high_h13,low_h13,open_h13,volumefrom_h13,volumeto_h13,date_h13,date_utc_h13,close_h14,high_h14,low_h14,open_h14,volumefrom_h14,volumeto_h14,date_h14,date_utc_h14,close_h15,high_h15,low_h15,open_h15,volumefrom_h15,volumeto_h15,date_h15,date_utc_h15,close_h16,high_h16,low_h16,open_h16,volumefrom_h16,volumeto_h16,date_h16,date_utc_h16,close_h17,high_h17,low_h17,open_h17,volumefrom_h17,volumeto_h17,date_h17,date_utc_h17,close_h18,high_h18,low_h18,open_h18,volumefrom_h18,volumeto_h18,date_h18,date_utc_h18,close_h19,high_h19,low_h19,open_h19,volumefrom_h19,volumeto_h19,date_h19,date_utc_h19,close_h20,high_h20,low_h20,open_h20,volumefrom_h20,volumeto_h20,date_h20,date_utc_h20,close_h21,high_h21,low_h21,open_h21,volumefrom_h21,volumeto_h21,date_h21,date_utc_h21,close_h22,high_h22,low_h22,open_h22,volumefrom_h22,volumeto_h22,date_h22,date_utc_h22,close_h23,high_h23,low_h23,open_h23,volumefrom_h23,volumeto_h23,date_h23,date_utc_h23,close_h24,high_h24,low_h24,open_h24,volumefrom_h24,volumeto_h24,date_h24,date_utc_h24,close_h25,high_h25,low_h25,open_h25,volumefrom_h25,volumeto_h25,date_h25,date_utc_h25,close_h26,high_h26,low_h26,open_h26,volumefrom_h26,volumeto_h26,date_h26,date_utc_h26,close_h27,high_h27,low_h27,open_h27,volumefrom_h27,volumeto_h27,date_h27,date_utc_h27,close_h28,high_h28,low_h28,open_h28,volumefrom_h28,volumeto_h28,date_h28,date_utc_h28,close_h29,high_h29,low_h29,open_h29,volumefrom_h29,volumeto_h29,date_h29,date_utc_h29,close_h30,high_h30,low_h30,open_h30,volumefrom_h30,volumeto_h30,date_h30,date_utc_h30,close_h31,high_h31,low_h31,open_h31,volumefrom_h31,volumeto_h31,date_h31,date_utc_h31,close_h32,high_h32,low_h32,open_h32,volumefrom_h32,volumeto_h32,date_h32,date_utc_h32,close_h33,high_h33,low_h33,open_h33,volumefrom_h33,volumeto_h33,date_h33,date_utc_h33,close_h34,high_h34,low_h34,open_h34,volumefrom_h34,volumeto_h34,date_h34,date_utc_h34,close_h35,high_h35,low_h35,open_h35,volumefrom_h35,volumeto_h35,date_h35,date_utc_h35,close_h36,high_h36,low_h36,open_h36,volumefrom_h36,volumeto_h36,date_h36,date_utc_h36,close_h37,high_h37,low_h37,open_h37,volumefrom_h37,volumeto_h37,date_h37,date_utc_h37,close_h38,high_h38,low_h38,open_h38,volumefrom_h38,volumeto

In [55]:
col_dict = dict()
for f in ['open','high','low','close','volumefrom','volumeto']:
    col_dict[f] = [f+"_h"+str(h) for h in range(num_hours + 1)]

In [56]:
# col_dict

In [57]:
# core = ['channelLink','channelTitle','currency','exchange',
#         'timestamp','signalTime','pumptime_edited','coin_name']
core = ['currency','exchange','pumptime_edited','coin_name','timestamp','event_date','event_time']
col_dict['core'] = core

In [58]:
col_dict.keys()

dict_keys(['open', 'high', 'low', 'close', 'volumefrom', 'volumeto', 'core'])

In [59]:
ohlcv_vars = []
for k,v in col_dict.items():
#     if k not in ['core','volumefrom']:
        ohlcv_vars+=col_dict[k]

## Make sure all the importmnat columns are preserved

In [60]:
len(ohlcv_vars)

301

In [61]:
orig_cols = x_final.columns
for i in orig_cols:
    if i not in ohlcv_vars:
        print(i)

timestamp_h0
timestamp_h1
timestamp_h2
timestamp_h3
timestamp_h4
timestamp_h5
timestamp_h6
timestamp_h7
timestamp_h8
timestamp_h9
timestamp_h10
timestamp_h11
timestamp_h12
timestamp_h13
timestamp_h14
timestamp_h15
timestamp_h16
timestamp_h17
timestamp_h18
timestamp_h19
timestamp_h20
timestamp_h21
timestamp_h22
timestamp_h23
timestamp_h24
timestamp_h25
timestamp_h26
timestamp_h27
timestamp_h28
timestamp_h29
timestamp_h30
timestamp_h31
timestamp_h32
timestamp_h33
timestamp_h34
timestamp_h35
timestamp_h36
timestamp_h37
timestamp_h38
timestamp_h39
timestamp_h40
timestamp_h41
timestamp_h42
timestamp_h43
timestamp_h44
timestamp_h45
timestamp_h46
timestamp_h47
timestamp_h48
date_h0
date_utc_h0
date_h1
date_utc_h1
date_h2
date_utc_h2
date_h3
date_utc_h3
date_h4
date_utc_h4
date_h5
date_utc_h5
date_h6
date_utc_h6
date_h7
date_utc_h7
date_h8
date_utc_h8
date_h9
date_utc_h9
date_h10
date_utc_h10
date_h11
date_utc_h11
date_h12
date_utc_h12
date_h13
date_utc_h13
date_h14
date_utc_h14
date_h15
date_

In [62]:
x_final = x_final[ohlcv_vars]
x_final = x_final.reset_index(drop=True)

In [63]:
x_final.head(2)

,open_h0,open_h1,open_h2,open_h3,open_h4,open_h5,open_h6,open_h7,open_h8,open_h9,open_h10,open_h11,open_h12,open_h13,open_h14,open_h15,open_h16,open_h17,open_h18,open_h19,open_h20,open_h21,open_h22,open_h23,open_h24,open_h25,open_h26,open_h27,open_h28,open_h29,open_h30,open_h31,open_h32,open_h33,open_h34,open_h35,open_h36,open_h37,open_h38,open_h39,open_h40,open_h41,open_h42,open_h43,open_h44,open_h45,open_h46,open_h47,open_h48,high_h0,high_h1,high_h2,high_h3,high_h4,high_h5,high_h6,high_h7,high_h8,high_h9,high_h10,high_h11,high_h12,high_h13,high_h14,high_h15,high_h16,high_h17,high_h18,high_h19,high_h20,high_h21,high_h22,high_h23,high_h24,high_h25,high_h26,high_h27,high_h28,high_h29,high_h30,high_h31,high_h32,high_h33,high_h34,high_h35,high_h36,high_h37,high_h38,high_h39,high_h40,high_h41,high_h42,high_h43,high_h44,high_h45,high_h46,high_h47,high_h48,low_h0,low_h1,low_h2,low_h3,low_h4,low_h5,low_h6,low_h7,low_h8,low_h9,low_h10,low_h11,low_h12,low_h13,low_h14,low_h15,low_h16,low_h17,low_h18,low_h19,low_h20,low_h21,low_h22,low_h23,low_h24,low_h25,low_h26,low_h27,low_h28,low_h29,low_h30,low_h31,low_h32,low_h33,low_h34,low_h35,low_h36,low_h37,low_h38,low_h39,low_h40,low_h41,low_h42,low_h43,low_h44,low_h45,low_h46,low_h47,low_h48,close_h0,close_h1,close_h2,close_h3,close_h4,close_h5,close_h6,close_h7,close_h8,close_h9,close_h10,close_h11,close_h12,close_h13,close_h14,close_h15,close_h16,close_h17,close_h18,close_h19,close_h20,close_h21,close_h22,close_h23,close_h24,close_h25,close_h26,close_h27,close_h28,close_h29,close_h30,close_h31,close_h32,close_h33,close_h34,close_h35,close_h36,close_h37,close_h38,close_h39,close_h40,close_h41,close_h42,close_h43,close_h44,close_h45,close_h46,close_h47,close_h48,volumefrom_h0,volumefrom_h1,volumefrom_h2,volumefrom_h3,volumefrom_h4,volumefrom_h5,volumefrom_h6,volumefrom_h7,volumefrom_h8,volumefrom_h9,volumefrom_h10,volumefrom_h11,volumefrom_h12,volumefrom_h13,volumefrom_h14,volumefrom_h15,volumefrom_h16,volumefrom_h17,volumefrom_h18,volumefrom_h19,volumefrom_h20,volumefrom_h21,volumefrom_h22,volumefrom_h23,volumefrom_h24,volumefrom_h25,volumefrom_h26,volumefrom_h27,volumefrom_h28,volumefrom_h29,volumefrom_h30,volumefrom_h31,volumefrom_h32,volumefrom_h33,volumefrom_h34,volumefrom_h35,volumefrom_h36,volumefrom_h37,volumefrom_h38,volumefrom_h39,volumefrom_h40,volumefrom_h41,volumefrom_h42,volumefrom_h43,volumefrom_h44,volumefrom_h45,volumefrom_h46,volumefrom_h47,volumefrom_h48,volumeto_h0,volumeto_h1,volumeto_h2,volumeto_h3,volumeto_h4,volumeto_h5,volumeto_h6,volumeto_h7,volumeto_h8,volumeto_h9,volumeto_h10,volumeto_h11,volumeto_h12,volumeto_h13,volumeto_h14,volumeto_h15,volumeto_h16,volumeto_h17,volumeto_h18,volumeto_h19,volumeto_h20,volumeto_h21,volumeto_h22,volumeto_h23,volumeto_h24,volumeto_h25,volumeto_h26,volumeto_h27,volumeto_h28,volumeto_h29,volumeto_h30,volumeto_h31,volumeto_h32,volumeto_h33,volumeto_h34,volumeto_h35,volumeto_h36,volumeto_h37,volumeto_h38,volumeto_h39,volumeto_h40,volumeto_h41,volumeto_h42,volumeto_h43,volumeto_h44,volumeto_h45,volumeto_h46,volumeto_h47,volumeto_h48,currency,exchange,pumptime_edited,coin_name,timestamp,event_date,event_time
0,0.000017,0.000017,0.000017,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000017,0.000017,0.000018,0.000017,0.000018,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000018,0.000018,0.000018,0.000017,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,0.000017,0.000018,0.000018,0.000018,0.000018,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017,0.000017

In [64]:
x_final.to_pickle("ohlcv_data_training_h.pkl")

### Also save the column names just to be sure


In [65]:
f = open("ohlcv_dict_h.pkl","wb")
pickle.dump(col_dict,f)
f.close()

g = open("ohlcv_features_h.pkl","wb")
pickle.dump(ohlcv_vars,g)
g.close()

In [66]:
ohlcv_vars

['open_h0',
 'open_h1',
 'open_h2',
 'open_h3',
 'open_h4',
 'open_h5',
 'open_h6',
 'open_h7',
 'open_h8',
 'open_h9',
 'open_h10',
 'open_h11',
 'open_h12',
 'open_h13',
 'open_h14',
 'open_h15',
 'open_h16',
 'open_h17',
 'open_h18',
 'open_h19',
 'open_h20',
 'open_h21',
 'open_h22',
 'open_h23',
 'open_h24',
 'open_h25',
 'open_h26',
 'open_h27',
 'open_h28',
 'open_h29',
 'open_h30',
 'open_h31',
 'open_h32',
 'open_h33',
 'open_h34',
 'open_h35',
 'open_h36',
 'open_h37',
 'open_h38',
 'open_h39',
 'open_h40',
 'open_h41',
 'open_h42',
 'open_h43',
 'open_h44',
 'open_h45',
 'open_h46',
 'open_h47',
 'open_h48',
 'high_h0',
 'high_h1',
 'high_h2',
 'high_h3',
 'high_h4',
 'high_h5',
 'high_h6',
 'high_h7',
 'high_h8',
 'high_h9',
 'high_h10',
 'high_h11',
 'high_h12',
 'high_h13',
 'high_h14',
 'high_h15',
 'high_h16',
 'high_h17',
 'high_h18',
 'high_h19',
 'high_h20',
 'high_h21',
 'high_h22',
 'high_h23',
 'high_h24',
 'high_h25',
 'high_h26',
 'high_h27',
 'high_h28',
 'high